In [ ]:
# try this next: 

In [ ]:


from sklearn.model_selection import GridSearchCV, train_test_split

from sklearn.ensemble import RandomForestRegressor

rfgs_parameters = {
    'n_estimators': [3000],
    'max_depth'   : [2,3,4],
    'max_features': ['sqrt'],
    "min_samples_split": [3,5,10,15],
    "min_samples_leaf": [3,5,10,15]
}

rfr_cv = GridSearchCV(RandomForestRegressor(), rfgs_parameters, cv=8, scoring= 'neg_mean_squared_log_error')

rfr_cv.fit(train_features, train_target)

print("RFR GridSearch score: "+str(rfr_cv.best_score_))

print("RFR GridSearch params: ")

print(rfr_cv.best_params_)

RFR GridSearch score: -0.03649769729374987

RFR GridSearch params: 
{'max_depth': 4, 'max_features': 'sqrt', 'min_samples_leaf': 10, 'min_samples_split': 10, 'n_estimators': 3000}



<br>

**Critical Thoughts:**

Found: the most important hyperparameter to be min samples leaf, followed by max features and number of estimators

<br>

In [136]:
# Tom Bresee
# Model: ........... Random Forest Regressor
# Data: ............ Full dataset (including location features)
# Goals ............ Gridsearch, RandomCVsearch, Optuna, Advanced Tuning

<br>

In [137]:
# this notebook rendered in .html:

<br>

*Libraries:*

In [138]:
# --------------------------------------------------------------------------
from mycreds import creds_tom
# --------------------------------------------------------------------------
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
# --------------------------------------------------------------------------
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format) 
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.pyplot as plt
# --------------------------------------------------------------------------
import glob  
from datetime import datetime as dt
import pandas_profiling
from pandas_profiling import ProfileReport
# from bokeh.layouts import column
# from bokeh.models import Div, TeX
# from bokeh.plotting import figure, show
# from bokeh.io import output_notebook
# --------------------------------------------------------------------------
import fiona
from sqlalchemy import create_engine
# output_notebook() # bokeh
# --------------------------------------------------------------------------
from sklearn.model_selection import train_test_split, KFold, cross_val_score 
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures
from sklearn.preprocessing import RobustScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_absolute_error, recall_score
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import explained_variance_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
import yellowbrick
from yellowbrick.regressor import PredictionError
from yellowbrick.regressor import ResidualsPlot
import xgboost as xgb
cm = sns.light_palette("blue", as_cmap=True)
# correlations.style.set_caption('Correlation Values Ranked').background_gradient(cmap=cm
from pprint import pprint
# import shap
# shap.initjs()

<br>

*Reading in the data (includes geo loc features):*

In [139]:
df_train = pd.read_pickle('train_df.pkl')  
df_val   = pd.read_pickle('validate_df.pkl')
df_test  = pd.read_pickle('test_df.pkl')

In [140]:
len(df_train), len(df_val), len(df_test)

(268032, 75498, 96260)

<br>

*Quick checks:*

```
# add in storey_range_avg...
# add in if possible the mapping from town to Region
# 

storey_range_min                       int64
storey_range_max                       int64

n_rooms                                int64

floor_area_sqm                       float64

remaining_lease_years                  int64

resale_price_norm                    float64

price_per_sq_ft_norm                 float64
price_per_sq_ft_per_lease_yr_norm    float64
price_per_sq_m_norm                  float64
price_per_sq_m_per_lease_yr_norm     float64

num_preschools_400m                  float64
num_preschools_1km                   float64
num_primary_schools_1km              float64
score_primary_schools_1km            float64
num_primary_schools_2km              float64
score_primary_schools_2km            float64
num_secondary_schools_1km            float64
score_secondary_schools_1km          float64
num_secondary_schools_2km            float64
score_secondary_schools_2km          float64
num_healthcare_2km                   float64
num_healthcare_5km                   float64
nearest_healthcare_clinc             float64
nearest_police                       float64
nearest_fire                         float64
num_maj_parks_1km                    float64
num_maj_parks_2km                    float64
nearest_maj_park                     float64
num_min_parks_1km                    float64
num_min_parks_2km                    float64
nearest_min_park                     float64
num_waterbodies_100m                 float64
num_waterbodies_500m                 float64
nearest_waterbody                    float64
num_supermarket_1_5km                float64
nearest_supermarket                  float64
num_wetmarket_1_5km                  float64
nearest_wetmarket                    float64
nearest_expressway                   float64
nearest_expressway_entrance_exit     float64
num_busstops_100m                    float64
num_busstops_400m                    float64
nearest_busstop                      float64
num_taxistands_100m                  float64
num_taxistands_400m                  float64
nearest_taxistand                    float64
num_transit_stations_1_5km           float64
score_stations_1_5km                 float64
nearest_transit_station              float64
num_transit_exits_100m               float64
num_transit_exits_400m               float64
dist_to_nearest_transit_exit         float64
num_malls_1_5km                      float64
nearest_mall                         float64
num_hawker_1_5km                     float64
nearest_hawker                       float64

dist_to_central_business_district    float64
dist_to_holland_village              float64
dist_to_dempsey_hill                 float64
dist_to_serangoon_gardens_circus     float64
dist_to_orchard                      float64
dist_to_arab_street                  float64
dist_to_thomson_ridge                float64
dist_to_east_coast_park              float64
dist_to_bishan_park                  float64
dist_to_botanic_gardens              float64
dist_to_hort_park                    float64
dist_to_sentosa                      float64
dist_to_gardens_by_the_bay           float64
dist_to_con_area_id_1                float64
dist_to_con_area_id_2                float64

dist_to_con_area_id_250              float64
dist_to_con_area_id_251              float64
dist_to_con_area_id_252              float64
dist_to_con_area_id_253              float64
dist_to_con_area_id_254              float64

town_bedok                           float64
town_bishan                          float64
town_bukit_batok                     float64
town_bukit_merah                     float64
town_bukit_panjang                   float64
town_bukit_timah                     float64
town_central_area                    float64
town_choa_chu_kang                   float64
town_clementi                        float64
town_geylang                         float64
town_hougang                         float64
town_jurong_east                     float64
town_jurong_west                     float64
town_kallang_whampoa                 float64
town_lim_chu_kang                    float64
town_marine_parade                   float64
town_pasir_ris                       float64
town_punggol                         float64
town_queenstown                      float64
town_sembawang                       float64
town_sengkang                        float64
town_serangoon                       float64
town_tampines                        float64
town_toa_payoh                       float64
town_woodlands                       float64
town_yishun                          float64

flat_model_adjoined_flat             float64
flat_model_apartment                 float64
flat_model_dbss                      float64
flat_model_improved                  float64
flat_model_improved_maisonette       float64
flat_model_maisonette                float64
flat_model_model_a                   float64
flat_model_model_a2                  float64
flat_model_model_a_maisonette        float64
flat_model_multi_generation          float64
flat_model_new_generation            float64
flat_model_premium_apartment         float64
flat_model_premium_apartment_loft    float64
flat_model_premium_maisonette        float64
flat_model_simplified                float64
flat_model_standard                  float64
flat_model_terrace                   float64
flat_model_type_s1                   float64
flat_model_type_s2                   float64

flat_type_2_room                     float64
flat_type_3_room                     float64
flat_type_4_room                     float64
flat_type_5_room                     float64
flat_type_executive                  float64
flat_type_multi_generation           float64
```

In [141]:
# df_train.dtypes

In [142]:
df_train.head(1).T  

,0
storey_range_min,1.00
storey_range_max,3.00
n_rooms,5.00
floor_area_sqm,110.00
remaining_lease_years,90.00
resale_price_norm,484204.45
price_per_sq_ft_norm,1341.69
price_per_sq_ft_per_lease_yr_norm,14.91
price_per_sq_m_norm,4401.86
price_per_sq_m_per_lease_yr_norm,48.91


In [143]:
len(df_train), len(df_val), len(df_test)

(268032, 75498, 96260)

In [144]:
sorted(df_train.columns)

['dist_to_arab_street',
 'dist_to_bishan_park',
 'dist_to_botanic_gardens',
 'dist_to_central_business_district',
 'dist_to_con_area_id_1',
 'dist_to_con_area_id_10',
 'dist_to_con_area_id_100',
 'dist_to_con_area_id_101',
 'dist_to_con_area_id_102',
 'dist_to_con_area_id_103',
 'dist_to_con_area_id_104',
 'dist_to_con_area_id_105',
 'dist_to_con_area_id_106',
 'dist_to_con_area_id_107',
 'dist_to_con_area_id_108',
 'dist_to_con_area_id_109',
 'dist_to_con_area_id_11',
 'dist_to_con_area_id_110',
 'dist_to_con_area_id_111',
 'dist_to_con_area_id_112',
 'dist_to_con_area_id_113',
 'dist_to_con_area_id_114',
 'dist_to_con_area_id_115',
 'dist_to_con_area_id_116',
 'dist_to_con_area_id_117',
 'dist_to_con_area_id_118',
 'dist_to_con_area_id_119',
 'dist_to_con_area_id_12',
 'dist_to_con_area_id_120',
 'dist_to_con_area_id_121',
 'dist_to_con_area_id_122',
 'dist_to_con_area_id_123',
 'dist_to_con_area_id_124',
 'dist_to_con_area_id_125',
 'dist_to_con_area_id_126',
 'dist_to_con_area_id_1

<br><br><br>

## Random Forest Regressor Model

<br>

#### Intro

*Train, Tune, Output*

<br>

#### All Columns

```
['dist_to_arab_street',
 'dist_to_bishan_park',
 'dist_to_botanic_gardens',
 'dist_to_central_business_district',
 'dist_to_con_area_id_1',
 'dist_to_con_area_id_10',
 'dist_to_con_area_id_100',
 'dist_to_con_area_id_101',
 'dist_to_con_area_id_102',
 'dist_to_con_area_id_103',
 'dist_to_con_area_id_104',
 'dist_to_con_area_id_105',
 'dist_to_con_area_id_106',
 'dist_to_con_area_id_107',
 'dist_to_con_area_id_108',
 'dist_to_con_area_id_109',
 'dist_to_con_area_id_11',
 'dist_to_con_area_id_110',
 'dist_to_con_area_id_111',
 'dist_to_con_area_id_112',
 'dist_to_con_area_id_113',
 'dist_to_con_area_id_114',
 'dist_to_con_area_id_115',
 'dist_to_con_area_id_116',
 'dist_to_con_area_id_117',
 'dist_to_con_area_id_118',
 'dist_to_con_area_id_119',
 'dist_to_con_area_id_12',
 'dist_to_con_area_id_120',
 'dist_to_con_area_id_121',
 'dist_to_con_area_id_122',
 'dist_to_con_area_id_123',
 'dist_to_con_area_id_124',
 'dist_to_con_area_id_125',
 'dist_to_con_area_id_126',
 'dist_to_con_area_id_127',
 'dist_to_con_area_id_128',
 'dist_to_con_area_id_129',
 'dist_to_con_area_id_13',
 'dist_to_con_area_id_130',
 'dist_to_con_area_id_131',
 'dist_to_con_area_id_132',
 'dist_to_con_area_id_133',
 'dist_to_con_area_id_134',
 'dist_to_con_area_id_135',
 'dist_to_con_area_id_136',
 'dist_to_con_area_id_137',
 'dist_to_con_area_id_138',
 'dist_to_con_area_id_139',
 'dist_to_con_area_id_14',
 'dist_to_con_area_id_140',
 'dist_to_con_area_id_141',
 'dist_to_con_area_id_142',
 'dist_to_con_area_id_143',
 'dist_to_con_area_id_144',
 'dist_to_con_area_id_145',
 'dist_to_con_area_id_146',
 'dist_to_con_area_id_147',
 'dist_to_con_area_id_148',
 'dist_to_con_area_id_149',
 'dist_to_con_area_id_15',
 'dist_to_con_area_id_150',
 'dist_to_con_area_id_151',
 'dist_to_con_area_id_152',
 'dist_to_con_area_id_153',
 'dist_to_con_area_id_154',
 'dist_to_con_area_id_155',
 'dist_to_con_area_id_156',
 'dist_to_con_area_id_157',
 'dist_to_con_area_id_158',
 'dist_to_con_area_id_159',
 'dist_to_con_area_id_16',
 'dist_to_con_area_id_160',
 'dist_to_con_area_id_161',
 'dist_to_con_area_id_162',
 'dist_to_con_area_id_163',
 'dist_to_con_area_id_164',
 'dist_to_con_area_id_165',
 'dist_to_con_area_id_166',
 'dist_to_con_area_id_167',
 'dist_to_con_area_id_168',
 'dist_to_con_area_id_169',
 'dist_to_con_area_id_17',
 'dist_to_con_area_id_170',
 'dist_to_con_area_id_171',
 'dist_to_con_area_id_172',
 'dist_to_con_area_id_173',
 'dist_to_con_area_id_174',
 'dist_to_con_area_id_175',
 'dist_to_con_area_id_176',
 'dist_to_con_area_id_177',
 'dist_to_con_area_id_178',
 'dist_to_con_area_id_179',
 'dist_to_con_area_id_18',
 'dist_to_con_area_id_180',
 'dist_to_con_area_id_181',
 'dist_to_con_area_id_182',
 'dist_to_con_area_id_183',
 'dist_to_con_area_id_184',
 'dist_to_con_area_id_185',
 'dist_to_con_area_id_186',
 'dist_to_con_area_id_187',
 'dist_to_con_area_id_188',
 'dist_to_con_area_id_189',
 'dist_to_con_area_id_19',
 'dist_to_con_area_id_190',
 'dist_to_con_area_id_191',
 'dist_to_con_area_id_192',
 'dist_to_con_area_id_193',
 'dist_to_con_area_id_194',
 'dist_to_con_area_id_195',
 'dist_to_con_area_id_196',
 'dist_to_con_area_id_197',
 'dist_to_con_area_id_198',
 'dist_to_con_area_id_199',
 'dist_to_con_area_id_2',
 'dist_to_con_area_id_20',
 'dist_to_con_area_id_200',
 'dist_to_con_area_id_201',
 'dist_to_con_area_id_202',
 'dist_to_con_area_id_203',
 'dist_to_con_area_id_204',
 'dist_to_con_area_id_205',
 'dist_to_con_area_id_206',
 'dist_to_con_area_id_207',
 'dist_to_con_area_id_208',
 'dist_to_con_area_id_209',
 'dist_to_con_area_id_21',
 'dist_to_con_area_id_210',
 'dist_to_con_area_id_211',
 'dist_to_con_area_id_212',
 'dist_to_con_area_id_213',
 'dist_to_con_area_id_214',
 'dist_to_con_area_id_215',
 'dist_to_con_area_id_216',
 'dist_to_con_area_id_217',
 'dist_to_con_area_id_218',
 'dist_to_con_area_id_219',
 'dist_to_con_area_id_22',
 'dist_to_con_area_id_220',
 'dist_to_con_area_id_221',
 'dist_to_con_area_id_222',
 'dist_to_con_area_id_223',
 'dist_to_con_area_id_224',
 'dist_to_con_area_id_225',
 'dist_to_con_area_id_226',
 'dist_to_con_area_id_227',
 'dist_to_con_area_id_228',
 'dist_to_con_area_id_229',
 'dist_to_con_area_id_23',
 'dist_to_con_area_id_230',
 'dist_to_con_area_id_231',
 'dist_to_con_area_id_232',
 'dist_to_con_area_id_233',
 'dist_to_con_area_id_234',
 'dist_to_con_area_id_235',
 'dist_to_con_area_id_236',
 'dist_to_con_area_id_237',
 'dist_to_con_area_id_238',
 'dist_to_con_area_id_239',
 'dist_to_con_area_id_24',
 'dist_to_con_area_id_240',
 'dist_to_con_area_id_241',
 'dist_to_con_area_id_242',
 'dist_to_con_area_id_243',
 'dist_to_con_area_id_244',
 'dist_to_con_area_id_245',
 'dist_to_con_area_id_246',
 'dist_to_con_area_id_247',
 'dist_to_con_area_id_248',
 'dist_to_con_area_id_249',
 'dist_to_con_area_id_25',
 'dist_to_con_area_id_250',
 'dist_to_con_area_id_251',
 'dist_to_con_area_id_252',
 'dist_to_con_area_id_253',
 'dist_to_con_area_id_254',
 'dist_to_con_area_id_26',
 'dist_to_con_area_id_27',
 'dist_to_con_area_id_28',
 'dist_to_con_area_id_29',
 'dist_to_con_area_id_3',
 'dist_to_con_area_id_30',
 'dist_to_con_area_id_31',
 'dist_to_con_area_id_32',
 'dist_to_con_area_id_33',
 'dist_to_con_area_id_34',
 'dist_to_con_area_id_35',
 'dist_to_con_area_id_36',
 'dist_to_con_area_id_37',
 'dist_to_con_area_id_38',
 'dist_to_con_area_id_39',
 'dist_to_con_area_id_4',
 'dist_to_con_area_id_40',
 'dist_to_con_area_id_41',
 'dist_to_con_area_id_42',
 'dist_to_con_area_id_43',
 'dist_to_con_area_id_44',
 'dist_to_con_area_id_45',
 'dist_to_con_area_id_46',
 'dist_to_con_area_id_47',
 'dist_to_con_area_id_48',
 'dist_to_con_area_id_49',
 'dist_to_con_area_id_5',
 'dist_to_con_area_id_50',
 'dist_to_con_area_id_51',
 'dist_to_con_area_id_52',
 'dist_to_con_area_id_53',
 'dist_to_con_area_id_54',
 'dist_to_con_area_id_55',
 'dist_to_con_area_id_56',
 'dist_to_con_area_id_57',
 'dist_to_con_area_id_58',
 'dist_to_con_area_id_59',
 'dist_to_con_area_id_6',
 'dist_to_con_area_id_60',
 'dist_to_con_area_id_61',
 'dist_to_con_area_id_62',
 'dist_to_con_area_id_63',
 'dist_to_con_area_id_64',
 'dist_to_con_area_id_65',
 'dist_to_con_area_id_66',
 'dist_to_con_area_id_67',
 'dist_to_con_area_id_68',
 'dist_to_con_area_id_69',
 'dist_to_con_area_id_7',
 'dist_to_con_area_id_70',
 'dist_to_con_area_id_71',
 'dist_to_con_area_id_72',
 'dist_to_con_area_id_73',
 'dist_to_con_area_id_74',
 'dist_to_con_area_id_75',
 'dist_to_con_area_id_76',
 'dist_to_con_area_id_77',
 'dist_to_con_area_id_78',
 'dist_to_con_area_id_79',
 'dist_to_con_area_id_8',
 'dist_to_con_area_id_80',
 'dist_to_con_area_id_81',
 'dist_to_con_area_id_82',
 'dist_to_con_area_id_83',
 'dist_to_con_area_id_84',
 'dist_to_con_area_id_85',
 'dist_to_con_area_id_86',
 'dist_to_con_area_id_87',
 'dist_to_con_area_id_88',
 'dist_to_con_area_id_89',
 'dist_to_con_area_id_9',
 'dist_to_con_area_id_90',
 'dist_to_con_area_id_91',
 'dist_to_con_area_id_92',
 'dist_to_con_area_id_93',
 'dist_to_con_area_id_94',
 'dist_to_con_area_id_95',
 'dist_to_con_area_id_96',
 'dist_to_con_area_id_97',
 'dist_to_con_area_id_98',
 'dist_to_con_area_id_99',
 'dist_to_dempsey_hill',
 'dist_to_east_coast_park',
 'dist_to_gardens_by_the_bay',
 'dist_to_holland_village',
 'dist_to_hort_park',
 'dist_to_nearest_transit_exit',
 'dist_to_orchard',
 'dist_to_sentosa',
 'dist_to_serangoon_gardens_circus',
 'dist_to_thomson_ridge',
 
 
 'flat_model_adjoined_flat',
 'flat_model_apartment',
 'flat_model_dbss',
 'flat_model_improved',
 'flat_model_improved_maisonette',
 'flat_model_maisonette',
 'flat_model_model_a',
 'flat_model_model_a2',
 'flat_model_model_a_maisonette',
 'flat_model_multi_generation',
 'flat_model_new_generation',
 'flat_model_premium_apartment',
 'flat_model_premium_apartment_loft',
 'flat_model_premium_maisonette',
 'flat_model_simplified',
 'flat_model_standard',
 'flat_model_terrace',
 'flat_model_type_s1',
 'flat_model_type_s2',
 
 
 'flat_type_2_room',
 'flat_type_3_room',
 'flat_type_4_room',
 'flat_type_5_room',
 'flat_type_executive',
 'flat_type_multi_generation',
 
 
 'floor_area_sqm',
 
 'n_rooms',
 
 'nearest_busstop',
 'nearest_expressway',
 'nearest_expressway_entrance_exit',
 'nearest_fire',
 'nearest_hawker',
 'nearest_healthcare_clinc',
 'nearest_maj_park',
 'nearest_mall',
 'nearest_min_park',
 'nearest_police',
 'nearest_supermarket',
 'nearest_taxistand',
 'nearest_transit_station',
 'nearest_waterbody',
 'nearest_wetmarket',
 
 
 'num_busstops_100m',
 'num_busstops_400m',
 'num_hawker_1_5km',
 'num_healthcare_2km',
 'num_healthcare_5km',
 'num_maj_parks_1km',
 'num_maj_parks_2km',
 'num_malls_1_5km',
 'num_min_parks_1km',
 'num_min_parks_2km',
 'num_preschools_1km',
 'num_preschools_400m',
 'num_primary_schools_1km',
 'num_primary_schools_2km',
 'num_secondary_schools_1km',
 'num_secondary_schools_2km',
 'num_supermarket_1_5km',
 'num_taxistands_100m',
 'num_taxistands_400m',
 'num_transit_exits_100m',
 'num_transit_exits_400m',
 'num_transit_stations_1_5km',
 'num_waterbodies_100m',
 'num_waterbodies_500m',
 'num_wetmarket_1_5km',
 
 
 'price_per_sq_ft_norm',
 'price_per_sq_ft_per_lease_yr_norm',
 'price_per_sq_m_norm',
 'price_per_sq_m_per_lease_yr_norm',
 
 
 'remaining_lease_years',
  
 'resale_price_norm',
 
 
 'score_primary_schools_1km',
 'score_primary_schools_2km',
 'score_secondary_schools_1km',
 'score_secondary_schools_2km',
 'score_stations_1_5km',
 
 
 'storey_range_max',
 'storey_range_min',
 
 
 'town_bedok',
 'town_bishan',
 'town_bukit_batok',
 'town_bukit_merah',
 'town_bukit_panjang',
 'town_bukit_timah',
 'town_central_area',
 'town_choa_chu_kang',
 'town_clementi',
 'town_geylang',
 'town_hougang',
 'town_jurong_east',
 'town_jurong_west',
 'town_kallang_whampoa',
 'town_lim_chu_kang',
 'town_marine_parade',
 'town_pasir_ris',
 'town_punggol',
 'town_queenstown',
 'town_sembawang',
 'town_sengkang',
 'town_serangoon',
 'town_tampines',
 'town_toa_payoh',
 'town_woodlands',
 'town_yishun']
```

<br>

####  Unneeded Columns 

In [163]:

c2d = ['resale_price_norm',
       'price_per_sq_m_norm',
       'price_per_sq_ft_norm',
       'price_per_sq_ft_per_lease_yr_norm',
       #  'price_per_sq_m_per_lease_yr_norm'
       'flat_model_adjoined_flat',
       'flat_model_apartment',
       'flat_model_dbss',
       'flat_model_improved',
       'flat_model_improved_maisonette',
       'flat_model_maisonette',
       'flat_model_model_a',
       'flat_model_model_a2',
       'flat_model_model_a_maisonette',
       'flat_model_multi_generation',
       'flat_model_new_generation',
       'flat_model_premium_apartment',
       'flat_model_premium_apartment_loft',
       'flat_model_premium_maisonette',
       'flat_model_simplified',
       'flat_model_standard',
       'flat_model_terrace',
       'flat_model_type_s1',
       'flat_model_type_s2']
       

<br>


#### Features 

In [146]:

df_train['storey_range_avg'] = (df_train['storey_range_min'] + df_train['storey_range_max'])/2
df_val['storey_range_avg'] = (df_val['storey_range_min'] + df_val['storey_range_max'])/2
df_test['storey_range_avg'] = (df_test['storey_range_min'] + df_test['storey_range_max'])/2


In [147]:
len(df_train.columns)

375

In [148]:

model_features = list(set(list(df_train.columns)) - set(c2d))


In [149]:

len(model_features) + len(c2d)


375

In [150]:

len(df_train.columns)


375

In [151]:

assert (len(model_features) + len(c2d) == len(df_train.columns))


In [152]:

print('\nTotal Number of Input Features to Model:', len(model_features), '\n')



Total Number of Input Features to Model: 352 



In [157]:
sorted(model_features)

['dist_to_arab_street',
 'dist_to_bishan_park',
 'dist_to_botanic_gardens',
 'dist_to_central_business_district',
 'dist_to_con_area_id_1',
 'dist_to_con_area_id_10',
 'dist_to_con_area_id_100',
 'dist_to_con_area_id_101',
 'dist_to_con_area_id_102',
 'dist_to_con_area_id_103',
 'dist_to_con_area_id_104',
 'dist_to_con_area_id_105',
 'dist_to_con_area_id_106',
 'dist_to_con_area_id_107',
 'dist_to_con_area_id_108',
 'dist_to_con_area_id_109',
 'dist_to_con_area_id_11',
 'dist_to_con_area_id_110',
 'dist_to_con_area_id_111',
 'dist_to_con_area_id_112',
 'dist_to_con_area_id_113',
 'dist_to_con_area_id_114',
 'dist_to_con_area_id_115',
 'dist_to_con_area_id_116',
 'dist_to_con_area_id_117',
 'dist_to_con_area_id_118',
 'dist_to_con_area_id_119',
 'dist_to_con_area_id_12',
 'dist_to_con_area_id_120',
 'dist_to_con_area_id_121',
 'dist_to_con_area_id_122',
 'dist_to_con_area_id_123',
 'dist_to_con_area_id_124',
 'dist_to_con_area_id_125',
 'dist_to_con_area_id_126',
 'dist_to_con_area_id_1

In [158]:
assert ('price_per_sq_m_per_lease_yr_norm' in model_features)

<br>

#### Base Model

In [160]:
from sklearn.model_selection import KFold, cross_val_score 
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_absolute_error 
from sklearn.metrics import explained_variance_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate

In [161]:
# what the fk:

import time
start_time = time.time()

target = ['price_per_sq_m_per_lease_yr_norm']  # predict this

X_train =  df_train[model_features].values
X_val   =  df_val[model_features].values
X_test  =  df_test[model_features].values
    
y_train =  df_train[target].values.ravel()
y_val   =  df_val[target].values.ravel()
y_test  =  df_test[target].values.ravel()
    

In [167]:
sorted(df_train[model_features].columns)

['dist_to_arab_street',
 'dist_to_bishan_park',
 'dist_to_botanic_gardens',
 'dist_to_central_business_district',
 'dist_to_con_area_id_1',
 'dist_to_con_area_id_10',
 'dist_to_con_area_id_100',
 'dist_to_con_area_id_101',
 'dist_to_con_area_id_102',
 'dist_to_con_area_id_103',
 'dist_to_con_area_id_104',
 'dist_to_con_area_id_105',
 'dist_to_con_area_id_106',
 'dist_to_con_area_id_107',
 'dist_to_con_area_id_108',
 'dist_to_con_area_id_109',
 'dist_to_con_area_id_11',
 'dist_to_con_area_id_110',
 'dist_to_con_area_id_111',
 'dist_to_con_area_id_112',
 'dist_to_con_area_id_113',
 'dist_to_con_area_id_114',
 'dist_to_con_area_id_115',
 'dist_to_con_area_id_116',
 'dist_to_con_area_id_117',
 'dist_to_con_area_id_118',
 'dist_to_con_area_id_119',
 'dist_to_con_area_id_12',
 'dist_to_con_area_id_120',
 'dist_to_con_area_id_121',
 'dist_to_con_area_id_122',
 'dist_to_con_area_id_123',
 'dist_to_con_area_id_124',
 'dist_to_con_area_id_125',
 'dist_to_con_area_id_126',
 'dist_to_con_area_id_1

<br>

<br>

### Mark 1 

In [172]:
  
def m1(df_train, df_val, df_test,model_features):
    
    import time
    
    start_time = time.time()
    
    target = ['price_per_sq_m_per_lease_yr_norm']  # predict this
    
    X_train =  df_train[model_features]
    X_train = X_train.drop('price_per_sq_m_per_lease_yr_norm', 1)
    X_train = X_train.values
    
    X_test =  df_test[model_features]
    X_test =  X_test.drop('price_per_sq_m_per_lease_yr_norm', 1)
    X_test = X_test.values
    
    X_val =  df_val[model_features]
    X_val = X_val.drop('price_per_sq_m_per_lease_yr_norm', 1)
    X_val = X_val.values
    
    y_train =  df_train[target].values.ravel()
    y_val   =  df_val[target].values.ravel()
    y_test  =  df_test[target].values.ravel()
    
    r_s     =  42
    n_e     =  100
    n_j     =  -1 
    v       =  1
    
    # status:  no tuning yet, base regressor model, all features
    
    rfm = RandomForestRegressor(random_state=r_s, 
                                n_estimators=n_e,
                                verbose=v, 
                                n_jobs=n_j)   

    rfm.fit(X_train, y_train)

    y_pred_train =  rfm.predict(X_train)
    y_pred_test  =  rfm.predict(X_test)
    y_pred_val   =  rfm.predict(X_val)
    
    print('\n--- Test Set ---')
    print('Mean Absolute Error: ...', mean_absolute_error(y_test, y_pred_test))
    print("Mean Squared Error:..... %.2f" % mean_squared_error(y_test, y_pred_test))
    print('RMSE: ..................', np.sqrt(mean_squared_error(y_test, y_pred_test)))   
    print("Coeff of det (R^2):..... %.3f" % r2_score(y_test, y_pred_test))  
    
    print('\n--- Val Set ---')
    print('Mean Absolute Error: ...', mean_absolute_error(y_val, y_pred_val))
    print("Mean Squared Error:..... %.2f" % mean_squared_error(y_val, y_pred_val))
    print('RMSE: ..................', np.sqrt(mean_squared_error(y_val, y_pred_val)))   
    print("Coeff of det (R^2):..... %.3f" % r2_score(y_val, y_pred_val))  

    print('\n--- Train Set ---')
    print('Mean Absolute Error: ...', mean_absolute_error(y_train, y_pred_train))
    print("Mean Squared Error:..... %.2f" % mean_squared_error(y_train, y_pred_train))
    print('RMSE: ..................', np.sqrt(mean_squared_error(y_train, y_pred_train)))
    print("Coeff of det (R^2):..... %.3f" % r2_score(y_train, y_pred_train))  
    
    print('\n\nModel Hyperparameters Definitions:')
    pprint(rfm.get_params())
    
    print("\n\n--- %s seconds ---" % round((time.time() - start_time)))
    print('')


In [173]:
m1(df_train, df_val, df_test,model_features)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 13.1min finished
[Parallel(n_jobs=24)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   2 tasks      | elapsed:    0.1s
[Parallel(n_jobs=24)]: Done 100 out of 100 | elapsed:    1.3s finished
[Parallel(n_jobs=24)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=24)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 100 out of 100 | elapsed:    0.2s finished



--- Test Set ---
Mean Absolute Error: ... 6.572905827346935
Mean Squared Error:..... 79.63
RMSE: .................. 8.92382132552867
Coeff of det (R^2):..... 0.773

--- Val Set ---
Mean Absolute Error: ... 4.727955706681909
Mean Squared Error:..... 44.45
RMSE: .................. 6.666710079721741
Coeff of det (R^2):..... 0.888

--- Train Set ---
Mean Absolute Error: ... 1.1208736790736153
Mean Squared Error:..... 2.94
RMSE: .................. 1.713928790746477
Coeff of det (R^2):..... 0.990


Model Hyperparameters Definitions:
{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': -1,
 'oob_score': False,
 'random_state': 42,
 'verbose': 1,
 'warm_start': False}


--- 791 seconds ---



<br>

<br>

### Mark 2

```
trying the exact features calculated without features, i.e. the very best you could do with no location features, to get mini baseline ...


    rf = RandomForestRegressor(random_state=42, 
                               verbose=0, 
                               n_estimators=200,    # locked
                               bootstrap=True,      # locked
                               max_depth= 22,       # locked 
                               max_features= 18,    # locked
                               min_samples_leaf= 2, # locked
                               min_samples_split=2, # locked (could be 3,4)
                               # max_leaf_nodes=max_leaf_nodes, #None,
                               # comment out  max_samples=None,
                               # comment out: min_weight_fraction_leaf=0.0,
                               n_jobs=-1)  
                               
                               
```

In [175]:
  
def m1(df_train, df_val, df_test,model_features):
    
    import time
    
    start_time = time.time()
    
    target = ['price_per_sq_m_per_lease_yr_norm']  # predict this
    
    X_train =  df_train[model_features]
    X_train = X_train.drop('price_per_sq_m_per_lease_yr_norm', 1)
    X_train = X_train.values
    
    X_test =  df_test[model_features]
    X_test =  X_test.drop('price_per_sq_m_per_lease_yr_norm', 1)
    X_test = X_test.values
    
    X_val =  df_val[model_features]
    X_val = X_val.drop('price_per_sq_m_per_lease_yr_norm', 1)
    X_val = X_val.values
    
    y_train =  df_train[target].values.ravel()
    y_val   =  df_val[target].values.ravel()
    y_test  =  df_test[target].values.ravel()
       
   
    rfm = RandomForestRegressor(random_state=42, 
                                
                                n_estimators=50,
                                
                                verbose=1, 
                                
                                bootstrap = True, 
                                
                                max_features = 20, 
                                 
                                max_depth=30, 
                                                    
                                min_samples_leaf= 2, 
                               
                                min_samples_split=3,
                                
                                n_jobs=-1)   

    rfm.fit(X_train, y_train)

    y_pred_train =  rfm.predict(X_train)
    y_pred_test  =  rfm.predict(X_test)
    y_pred_val   =  rfm.predict(X_val)
    
    print('\n--- Test Set ---')
    print('Mean Absolute Error: ...', mean_absolute_error(y_test, y_pred_test))
    print("Mean Squared Error:..... %.2f" % mean_squared_error(y_test, y_pred_test))
    print('RMSE: ..................', np.sqrt(mean_squared_error(y_test, y_pred_test)))   
    print("Coeff of det (R^2):..... %.3f" % r2_score(y_test, y_pred_test))  
    
    print('\n--- Val Set ---')
    print('Mean Absolute Error: ...', mean_absolute_error(y_val, y_pred_val))
    print("Mean Squared Error:..... %.2f" % mean_squared_error(y_val, y_pred_val))
    print('RMSE: ..................', np.sqrt(mean_squared_error(y_val, y_pred_val)))   
    print("Coeff of det (R^2):..... %.3f" % r2_score(y_val, y_pred_val))  

    print('\n--- Train Set ---')
    print('Mean Absolute Error: ...', mean_absolute_error(y_train, y_pred_train))
    print("Mean Squared Error:..... %.2f" % mean_squared_error(y_train, y_pred_train))
    print('RMSE: ..................', np.sqrt(mean_squared_error(y_train, y_pred_train)))
    print("Coeff of det (R^2):..... %.3f" % r2_score(y_train, y_pred_train))  
    
    print('\n\nModel Hyperparameters Definitions:')
    pprint(rfm.get_params())
    
    print("\n\n--- %s seconds ---" % round((time.time() - start_time)))
    print('')


In [176]:
m1(df_train, df_val, df_test,model_features)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    4.2s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   13.3s finished
[Parallel(n_jobs=24)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done  50 out of  50 | elapsed:    0.3s finished
[Parallel(n_jobs=24)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done  50 out of  50 | elapsed:    0.0s finished
[Parallel(n_jobs=24)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done  50 out of  50 | elapsed:    0.0s finished



--- Test Set ---
Mean Absolute Error: ... 6.586907097878101
Mean Squared Error:..... 78.16
RMSE: .................. 8.840978063564851
Coeff of det (R^2):..... 0.777

--- Val Set ---
Mean Absolute Error: ... 5.16912750736304
Mean Squared Error:..... 51.99
RMSE: .................. 7.210505999660623
Coeff of det (R^2):..... 0.869

--- Train Set ---
Mean Absolute Error: ... 2.1729718607055606
Mean Squared Error:..... 9.30
RMSE: .................. 3.0498998693262136
Coeff of det (R^2):..... 0.969


Model Hyperparameters Definitions:
{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': 30,
 'max_features': 20,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 2,
 'min_samples_split': 3,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 50,
 'n_jobs': -1,
 'oob_score': False,
 'random_state': 42,
 'verbose': 1,
 'warm_start': False}


--- 15 seconds ---



<br><br><br>

<br><br><br>

### Mark 3

```
trying the exact features calculated without features, i.e. the very best you could do with no location features, to get mini baseline ...


    rf = RandomForestRegressor(random_state=42, 
                               verbose=0, 
                               n_estimators=200,    # locked
                               bootstrap=True,      # locked
                               max_depth= 22,       # locked 
                               max_features= 18,    # locked
                               min_samples_leaf= 2, # locked
                               min_samples_split=2, # locked (could be 3,4)
                               # max_leaf_nodes=max_leaf_nodes, #None,
                               # comment out  max_samples=None,
                               # comment out: min_weight_fraction_leaf=0.0,
                               n_jobs=-1)  
                               
                               
```

In [177]:
  
def m1(df_train, df_val, df_test,model_features, input_max_depth):
    
    import time
    
    start_time = time.time()
    
    target = ['price_per_sq_m_per_lease_yr_norm']  # predict this
    
    X_train =  df_train[model_features]
    X_train = X_train.drop('price_per_sq_m_per_lease_yr_norm', 1)
    X_train = X_train.values
    
    X_test =  df_test[model_features]
    X_test =  X_test.drop('price_per_sq_m_per_lease_yr_norm', 1)
    X_test = X_test.values
    
    X_val =  df_val[model_features]
    X_val = X_val.drop('price_per_sq_m_per_lease_yr_norm', 1)
    X_val = X_val.values
    
    y_train =  df_train[target].values.ravel()
    y_val   =  df_val[target].values.ravel()
    y_test  =  df_test[target].values.ravel()
       
   
    rfm = RandomForestRegressor(random_state=42, 
                                
                                n_estimators=100,
                                
                                verbose=1, 
                                
                                bootstrap = True, 
                                
                                max_features = 20, 
                                 
                                max_depth=input_max_depth,
                                                    
                                min_samples_leaf= 2, 
                               
                                min_samples_split=3,
                                
                                n_jobs=-1)   

    rfm.fit(X_train, y_train)

    y_pred_train =  rfm.predict(X_train)
    y_pred_test  =  rfm.predict(X_test)
    y_pred_val   =  rfm.predict(X_val)
    
    print('INPUT MAX DEPTH',  input_max_depth)
    print("")
    
    print('\n--- Test Set ---')
    print('Mean Absolute Error: ...', mean_absolute_error(y_test, y_pred_test))
    print("Mean Squared Error:..... %.2f" % mean_squared_error(y_test, y_pred_test))
    print('RMSE: ..................', np.sqrt(mean_squared_error(y_test, y_pred_test)))   
    print("Coeff of det (R^2):..... %.3f" % r2_score(y_test, y_pred_test))  
    
    print('\n--- Val Set ---')
    print('Mean Absolute Error: ...', mean_absolute_error(y_val, y_pred_val))
    print("Mean Squared Error:..... %.2f" % mean_squared_error(y_val, y_pred_val))
    print('RMSE: ..................', np.sqrt(mean_squared_error(y_val, y_pred_val)))   
    print("Coeff of det (R^2):..... %.3f" % r2_score(y_val, y_pred_val))  

    print('\n--- Train Set ---')
    print('Mean Absolute Error: ...', mean_absolute_error(y_train, y_pred_train))
    print("Mean Squared Error:..... %.2f" % mean_squared_error(y_train, y_pred_train))
    print('RMSE: ..................', np.sqrt(mean_squared_error(y_train, y_pred_train)))
    print("Coeff of det (R^2):..... %.3f" % r2_score(y_train, y_pred_train))  
    
    print('\n\nModel Hyperparameters Definitions:')
    pprint(rfm.get_params())
    
    print("\n\n--- %s seconds ---" % round((time.time() - start_time)))
    print('')


In [178]:

for i in [5,10,15,20,25,30,35,40]:
    m1(df_train, df_val, df_test,model_features, i)
    

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    5.6s finished
[Parallel(n_jobs=24)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=24)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=24)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 100 out of 100 | elapsed:    0.0s finished


INPUT MAX DEPTH 5


--- Test Set ---
Mean Absolute Error: ... 7.719683700211707
Mean Squared Error:..... 112.04
RMSE: .................. 10.584975593224373
Coeff of det (R^2):..... 0.681

--- Val Set ---
Mean Absolute Error: ... 7.019093317944223
Mean Squared Error:..... 95.22
RMSE: .................. 9.758042162923626
Coeff of det (R^2):..... 0.761

--- Train Set ---
Mean Absolute Error: ... 5.524511911602065
Mean Squared Error:..... 55.22
RMSE: .................. 7.4308003152935225
Coeff of det (R^2):..... 0.813


Model Hyperparameters Definitions:
{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': 5,
 'max_features': 20,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 2,
 'min_samples_split': 3,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': -1,
 'oob_score': False,
 'random_state': 42,
 'verbose': 1,
 'warm_start': False}


--- 7 seconds ---



[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   16.5s finished
[Parallel(n_jobs=24)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=24)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=24)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 100 out of 100 | elapsed:    0.0s finished


INPUT MAX DEPTH 10


--- Test Set ---
Mean Absolute Error: ... 7.147046925572245
Mean Squared Error:..... 92.33
RMSE: .................. 9.608975999995222
Coeff of det (R^2):..... 0.737

--- Val Set ---
Mean Absolute Error: ... 6.011572631601378
Mean Squared Error:..... 69.36
RMSE: .................. 8.328363426830363
Coeff of det (R^2):..... 0.826

--- Train Set ---
Mean Absolute Error: ... 3.9654242132691198
Mean Squared Error:..... 28.55
RMSE: .................. 5.3436135599127
Coeff of det (R^2):..... 0.903


Model Hyperparameters Definitions:
{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': 10,
 'max_features': 20,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 2,
 'min_samples_split': 3,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': -1,
 'oob_score': False,
 'random_state': 42,
 'verbose': 1,
 'warm_start': False}


--- 18 seconds ---



[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    5.4s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   27.0s finished
[Parallel(n_jobs=24)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 100 out of 100 | elapsed:    0.4s finished
[Parallel(n_jobs=24)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=24)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 100 out of 100 | elapsed:    0.0s finished


...

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    8.8s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   41.7s finished
[Parallel(n_jobs=24)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 100 out of 100 | elapsed:    0.8s finished
[Parallel(n_jobs=24)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=24)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 100 out of 100 | elapsed:    0.2s finished


INPUT MAX DEPTH 35


--- Test Set ---
Mean Absolute Error: ... 6.563331770238124
Mean Squared Error:..... 77.76
RMSE: .................. 8.818239361042496
Coeff of det (R^2):..... 0.778

--- Val Set ---
Mean Absolute Error: ... 5.140727710557873
Mean Squared Error:..... 51.43
RMSE: .................. 7.171208661624641
Coeff of det (R^2):..... 0.871

--- Train Set ---
Mean Absolute Error: ... 2.1545761473233727
Mean Squared Error:..... 9.15
RMSE: .................. 3.0243125019478962
Coeff of det (R^2):..... 0.969


Model Hyperparameters Definitions:
{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': 35,
 'max_features': 20,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 2,
 'min_samples_split': 3,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': -1,
 'oob_score': False,
 'random_state': 42,
 'verbose': 1,
 'warm_start': False}


--- 45 seconds ---



[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    8.2s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   40.8s finished
[Parallel(n_jobs=24)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 100 out of 100 | elapsed:    0.8s finished
[Parallel(n_jobs=24)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=24)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   2 tasks      | elapsed:    0.0s


INPUT MAX DEPTH 40


--- Test Set ---
Mean Absolute Error: ... 6.570366904681262
Mean Squared Error:..... 77.79
RMSE: .................. 8.819862332374127
Coeff of det (R^2):..... 0.778

--- Val Set ---
Mean Absolute Error: ... 5.149236288466931
Mean Squared Error:..... 51.58
RMSE: .................. 7.182049147844673
Coeff of det (R^2):..... 0.870

--- Train Set ---
Mean Absolute Error: ... 2.154665096736477
Mean Squared Error:..... 9.15
RMSE: .................. 3.024387851660816
Coeff of det (R^2):..... 0.969


Model Hyperparameters Definitions:
{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': 40,
 'max_features': 20,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 2,
 'min_samples_split': 3,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': -1,
 'oob_score': False,
 'random_state': 42,
 'verbose': 1,
 'warm_start': False}


--- 44 seconds ---



[Parallel(n_jobs=24)]: Done 100 out of 100 | elapsed:    0.2s finished


In [179]:
  
def m1_slim(df_train, df_val, df_test,model_features, input_max_depth):
    
    target = ['price_per_sq_m_per_lease_yr_norm']  # predict this
    X_train =  df_train[model_features]
    X_train = X_train.drop('price_per_sq_m_per_lease_yr_norm', 1)
    X_train = X_train.values
    
    X_test =  df_test[model_features]
    X_test =  X_test.drop('price_per_sq_m_per_lease_yr_norm', 1)
    X_test = X_test.values
    
    X_val =  df_val[model_features]
    X_val = X_val.drop('price_per_sq_m_per_lease_yr_norm', 1)
    X_val = X_val.values
    
    y_train =  df_train[target].values.ravel()
    y_val   =  df_val[target].values.ravel()
    y_test  =  df_test[target].values.ravel()
       
   
    rfm = RandomForestRegressor(random_state=42, 
                                
                                n_estimators=100,
                                
                                verbose=0, 
                                
                                bootstrap = True, 
                                
                                max_features = 20, 
                                 
                                max_depth=input_max_depth,
                                                    
                                min_samples_leaf= 2, 
                               
                                min_samples_split=3,
                                
                                n_jobs=-1)   

    rfm.fit(X_train, y_train)

    y_pred_train =  rfm.predict(X_train)
    y_pred_test  =  rfm.predict(X_test)
    y_pred_val   =  rfm.predict(X_val)
    
    print('INPUT MAX DEPTH',  input_max_depth)
    print("")
    
    print('\n--- Test Set ---')  
    print("Coeff of det (R^2):..... %.3f" % r2_score(y_test, y_pred_test))  
    
    print('\n--- Val Set ---')  
    print("Coeff of det (R^2):..... %.3f" % r2_score(y_val, y_pred_val))  

    print('\n--- Train Set ---')
    print("Coeff of det (R^2):..... %.3f" % r2_score(y_train, y_pred_train))  
    
    # print('\n\nModel Hyperparameters Definitions:')
    # pprint(rfm.get_params())
    
    # print("\n\n--- %s seconds ---" % round((time.time() - start_time)))
    # print('')


In [180]:

for i in [40, 45, 50, 55, 60, 65]:
    m1_slim(df_train, df_val, df_test,model_features, i)
    

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    4.2s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   31.0s finished
[Parallel(n_jobs=24)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 100 out of 100 | elapsed:    0.8s finished
[Parallel(n_jobs=24)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=24)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   2 tasks      | elapsed:    1.4s
[Parallel(n_jobs=24)]: Done 100 out of 100 | elapsed:    1.7s finished


INPUT MAX DEPTH 40


--- Test Set ---
Coeff of det (R^2):..... 0.778

--- Val Set ---
Coeff of det (R^2):..... 0.870

--- Train Set ---
Coeff of det (R^2):..... 0.969


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    9.4s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   43.1s finished
[Parallel(n_jobs=24)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 100 out of 100 | elapsed:    1.1s finished
[Parallel(n_jobs=24)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=24)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 100 out of 100 | elapsed:    0.2s finished


INPUT MAX DEPTH 45


--- Test Set ---
Coeff of det (R^2):..... 0.779

--- Val Set ---
Coeff of det (R^2):..... 0.870

--- Train Set ---
Coeff of det (R^2):..... 0.969


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    9.5s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   45.7s finished
[Parallel(n_jobs=24)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 100 out of 100 | elapsed:    0.9s finished
[Parallel(n_jobs=24)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=24)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 100 out of 100 | elapsed:    0.2s finished


...

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    9.0s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   44.2s finished
[Parallel(n_jobs=24)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 100 out of 100 | elapsed:    1.0s finished
[Parallel(n_jobs=24)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=24)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 100 out of 100 | elapsed:    0.2s finished


INPUT MAX DEPTH 60


--- Test Set ---
Coeff of det (R^2):..... 0.779

--- Val Set ---
Coeff of det (R^2):..... 0.870

--- Train Set ---
Coeff of det (R^2):..... 0.969


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    9.4s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   44.7s finished
[Parallel(n_jobs=24)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 100 out of 100 | elapsed:    1.2s finished
[Parallel(n_jobs=24)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=24)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   2 tasks      | elapsed:    0.0s


INPUT MAX DEPTH 65


--- Test Set ---
Coeff of det (R^2):..... 0.779

--- Val Set ---
Coeff of det (R^2):..... 0.870

--- Train Set ---
Coeff of det (R^2):..... 0.969


[Parallel(n_jobs=24)]: Done 100 out of 100 | elapsed:    0.2s finished


<br><br><br>

In [184]:

# iterate


def m1_slim(df_train, df_val, df_test,model_features, input_min_samples_split):
    
    target = ['price_per_sq_m_per_lease_yr_norm']  # predict this
    X_train =  df_train[model_features]
    X_train = X_train.drop('price_per_sq_m_per_lease_yr_norm', 1)
    X_train = X_train.values
    
    X_test =  df_test[model_features]
    X_test =  X_test.drop('price_per_sq_m_per_lease_yr_norm', 1)
    X_test = X_test.values
    
    X_val =  df_val[model_features]
    X_val = X_val.drop('price_per_sq_m_per_lease_yr_norm', 1)
    X_val = X_val.values
    
    y_train =  df_train[target].values.ravel()
    y_val   =  df_val[target].values.ravel()
    y_test  =  df_test[target].values.ravel()
       
   
    rfm = RandomForestRegressor(random_state=42, 
                                
                                n_estimators=100,
                                
                                verbose=0, 
                                
                                # bootstrap = True, 
                                
                                max_features = 'auto', 
                                 
                                max_depth= 20,   # 30 or 40 ? 
                                                    
                                min_samples_leaf= 2, 
                               
                                min_samples_split=input_min_samples_split,
                                
                                n_jobs=-1)   
                     
    rfm.fit(X_train, y_train)

    y_pred_train =  rfm.predict(X_train)
    y_pred_test  =  rfm.predict(X_test)
    y_pred_val   =  rfm.predict(X_val)

    print('Min Samples Split: ', input_min_samples_split)
    
    print('\n--- Test Set ---')  
    print("Coeff of det (R^2):..... %.3f" % r2_score(y_test, y_pred_test))  
    
    print('\n--- Val Set ---')  
    print("Coeff of det (R^2):..... %.3f" % r2_score(y_val, y_pred_val))  

    print('\n--- Train Set ---')
    print("Coeff of det (R^2):..... %.3f" % r2_score(y_train, y_pred_train))  
    
    # print('\n\nModel Hyperparameters Definitions:')
    # pprint(rfm.get_params())
    
    # print("\n\n--- %s seconds ---" % round((time.time() - start_time)))
    # print('')

    
for i in [2,4,6,8,10,12,14,16]:
    m1_slim(df_train, df_val, df_test,model_features, i)
    


Min Samples Split:  2

--- Test Set ---
Coeff of det (R^2):..... 0.774

--- Val Set ---
Coeff of det (R^2):..... 0.887

--- Train Set ---
Coeff of det (R^2):..... 0.982



KeyboardInterrupt



In [187]:

def m1_slim(df_train, df_val, df_test,model_features):
    
    target = ['price_per_sq_m_per_lease_yr_norm']  # predict this
    X_train =  df_train[model_features]
    X_train = X_train.drop('price_per_sq_m_per_lease_yr_norm', 1)
    X_train = X_train.values
    
    X_test =  df_test[model_features]
    X_test =  X_test.drop('price_per_sq_m_per_lease_yr_norm', 1)
    X_test = X_test.values
    
    X_val =  df_val[model_features]
    X_val = X_val.drop('price_per_sq_m_per_lease_yr_norm', 1)
    X_val = X_val.values
    
    y_train =  df_train[target].values.ravel()
    y_val   =  df_val[target].values.ravel()
    y_test  =  df_test[target].values.ravel()
       
        
    # {'max_depth': 20, 'max_features': 'auto', 'min_samples_leaf': 2, 'min_samples_split': 15}
    
   
    rfm = RandomForestRegressor(random_state=42, 
                                
                                n_estimators=100,
                                
                                verbose=0, 
                                
                                # bootstrap = True, 
                                
                                max_features = 'auto', 
                                 
                                max_depth= 20, 
                                                    
                                min_samples_leaf= 2, 
                                
                                min_samples_split= 15,
                                
                                n_jobs=-1)   
                     
    rfm.fit(X_train, y_train)

    y_pred_train =  rfm.predict(X_train)
    y_pred_test  =  rfm.predict(X_test)
    y_pred_val   =  rfm.predict(X_val)

    # print('Min Samples Split: ', input_min_samples_split)
    
    print('\n--- Test Set ---')  
    print("Coeff of det (R^2):..... %.3f" % r2_score(y_test, y_pred_test))  
    
    print('\n--- Val Set ---')  
    print("Coeff of det (R^2):..... %.3f" % r2_score(y_val, y_pred_val))  

    print('\n--- Train Set ---')
    print("Coeff of det (R^2):..... %.3f" % r2_score(y_train, y_pred_train))  
    
    # print('\n\nModel Hyperparameters Definitions:')
    # pprint(rfm.get_params())
    
    # print("\n\n--- %s seconds ---" % round((time.time() - start_time)))
    # print('')

    
m1_slim(df_train, df_val, df_test,model_features)
    



--- Test Set ---
Coeff of det (R^2):..... 0.772

--- Val Set ---
Coeff of det (R^2):..... 0.885

--- Train Set ---
Coeff of det (R^2):..... 0.975


In [188]:

def m1_slim(df_train, df_val, df_test,model_features):
    
    target = ['price_per_sq_m_per_lease_yr_norm']  # predict this
    X_train =  df_train[model_features]
    X_train = X_train.drop('price_per_sq_m_per_lease_yr_norm', 1)
    X_train = X_train.values
    
    X_test =  df_test[model_features]
    X_test =  X_test.drop('price_per_sq_m_per_lease_yr_norm', 1)
    X_test = X_test.values
    
    X_val =  df_val[model_features]
    X_val = X_val.drop('price_per_sq_m_per_lease_yr_norm', 1)
    X_val = X_val.values
    
    y_train =  df_train[target].values.ravel()
    y_val   =  df_val[target].values.ravel()
    y_test  =  df_test[target].values.ravel()
       
        
    # {'max_depth': 20, 'max_features': 'auto', 'min_samples_leaf': 2, 'min_samples_split': 15}
    
   
    rfm = RandomForestRegressor(random_state=42, 
                                
                                n_estimators=200,
                                
                                verbose=0, 
                                
                                # bootstrap = True, 
                                
                                max_features = 'auto',   # 
                                 
                                max_depth= 40, 
                                                    
                                min_samples_leaf= 2, 
                                
                                min_samples_split= 10,
                                
                                n_jobs=-1)   
                     
    rfm.fit(X_train, y_train)

    y_pred_train =  rfm.predict(X_train)
    y_pred_test  =  rfm.predict(X_test)
    y_pred_val   =  rfm.predict(X_val)

    # print('Min Samples Split: ', input_min_samples_split)
    
    print('\n--- Test Set ---')  
    print("Coeff of det (R^2):..... %.3f" % r2_score(y_test, y_pred_test))  
    
    print('\n--- Val Set ---')  
    print("Coeff of det (R^2):..... %.3f" % r2_score(y_val, y_pred_val))  

    print('\n--- Train Set ---')
    print("Coeff of det (R^2):..... %.3f" % r2_score(y_train, y_pred_train))  
    
    # print('\n\nModel Hyperparameters Definitions:')
    # pprint(rfm.get_params())
    
    # print("\n\n--- %s seconds ---" % round((time.time() - start_time)))
    # print('')

    
m1_slim(df_train, df_val, df_test,model_features)



--- Test Set ---
Coeff of det (R^2):..... 0.775

--- Val Set ---
Coeff of det (R^2):..... 0.887

--- Train Set ---
Coeff of det (R^2):..... 0.980


In [189]:

def m1_slim(df_train, df_val, df_test,model_features):
    
    target = ['price_per_sq_m_per_lease_yr_norm']  # predict this
    X_train =  df_train[model_features]
    X_train = X_train.drop('price_per_sq_m_per_lease_yr_norm', 1)
    X_train = X_train.values
    
    X_test =  df_test[model_features]
    X_test =  X_test.drop('price_per_sq_m_per_lease_yr_norm', 1)
    X_test = X_test.values
    
    X_val =  df_val[model_features]
    X_val = X_val.drop('price_per_sq_m_per_lease_yr_norm', 1)
    X_val = X_val.values
    
    y_train =  df_train[target].values.ravel()
    y_val   =  df_val[target].values.ravel()
    y_test  =  df_test[target].values.ravel()
       
        
    # {'max_depth': 20, 'max_features': 'auto', 'min_samples_leaf': 2, 'min_samples_split': 15}
    
   
    rfm = RandomForestRegressor(random_state=42, 
                                
                                n_estimators=50,
                                
                                verbose=0, 
                                
                                # bootstrap = True, 
                                
                                max_features = 'auto',   # 
                                 
                                max_depth= 20, 
                                                    
                                min_samples_leaf= 2, 
                                
                                min_samples_split= 5,
                                
                                n_jobs=-1)   
                     
    rfm.fit(X_train, y_train)

    y_pred_train =  rfm.predict(X_train)
    y_pred_test  =  rfm.predict(X_test)
    y_pred_val   =  rfm.predict(X_val)

    # print('Min Samples Split: ', input_min_samples_split)
    
    print('\n--- Test Set ---')  
    print("Coeff of det (R^2):..... %.3f" % r2_score(y_test, y_pred_test))  
    
    print('\n--- Val Set ---')  
    print("Coeff of det (R^2):..... %.3f" % r2_score(y_val, y_pred_val))  

    print('\n--- Train Set ---')
    print("Coeff of det (R^2):..... %.3f" % r2_score(y_train, y_pred_train))  
    
    # print('\n\nModel Hyperparameters Definitions:')
    # pprint(rfm.get_params())
    
    # print("\n\n--- %s seconds ---" % round((time.time() - start_time)))
    # print('')

    
m1_slim(df_train, df_val, df_test,model_features)



--- Test Set ---
Coeff of det (R^2):..... 0.773

--- Val Set ---
Coeff of det (R^2):..... 0.886

--- Train Set ---
Coeff of det (R^2):..... 0.981


<br><br><br>